# Functions for creating annotated dataset for binary sequence classification.
## The sequence in this case is a sentence, and it is a positive sample if deemed relevant in the data pre-processing

In [1]:
# necessary library imports
import pandas as pd
import numpy as np

In [2]:
df_train_cleaned = pd.read_pickle("../data_frames/df_train_cleaned.pkl")
df_relevant_sentences = pd.read_pickle("../data_frames/df_relevant_sentences_lemma_stop.pkl")

In [10]:
# create the dataset with the corresponding labels
def label_data(df, df_relevant_sentences):
    data_map = {}
    num_removed = 0
    for index, row in df.iterrows():
        sentences = row['context_raw']
        answer = row['correct_answer_raw']
        sent_with_ans_id = row['answer_location']
        relevant_sentence_ids = df_relevant_sentences.iloc[index]['ranked_matching_sentence_ids']
        sent_ids = [sent_with_ans_id]
        
        count = 0
        # add (max 3 including sentence with answer) highest ranked sententces
        for sent_id in relevant_sentence_ids:
            if count < 2 and sent_id != sent_with_ans_id:
                sent_ids.append(sent_id)
                count += 1
        
        # label all sentences and add to map
        for idx, sent in enumerate(sentences):
            key = ' '.join(sent)
            label = 0
            if idx in sent_ids: # sentence is relevant!
                label = 1
            if key in data_map:
                if label > 0:
                    data_map[key]['label'] = label
            else:
                data_point = {'context_nr': index, 'label': label, 'sentence': sent }
                data_map[key] = data_point
                
    
    # format labels and add answer labels
    for v in data_map.values():
        v['label'] = int(v['label'])
    labeled_data = list(data_map.values())
    print('num data points: ', len(labeled_data))
    return labeled_data

In [11]:
labeled_data = label_data(df_train_cleaned, df_relevant_sentences)
labeled_df = pd.DataFrame(labeled_data)
labeled_df.to_pickle("./data/labeled_sentence_classification_training_data.pkl")
labeled_df.head()

num data points:  8280


,context_nr,label,sentence
0,0,0,"[Visum, för, affärs-, och, konferensbesök]"
1,0,1,"[Om, du, ska, besöka, Sverige, för, ett, affär..."
2,0,1,"[Visum, är, ett, tillstånd, för, att, resa, in..."
3,0,0,"[För, att, få, visum, behöver, du, en, inbjuda..."
4,0,1,"[Du, ska, visa, att, du, har, pengar, för, dit..."
